<a href="https://colab.research.google.com/github/Lior-Baruch/SFT_llama_qlora/blob/main/DPO_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install all dependencies
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
# all imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import warnings

from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, AutoPeftModelForCausalLM
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer, DPOTrainer

warnings.filterwarnings("ignore")


In [3]:
from huggingface_hub import notebook_login

# log in to the Hugging Face hub (required for private datasets/models)
notebook_login()

In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

LoRA adapter

In [5]:
# Define LoRA ("low-rank attention") config
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

load our saved model from the hub

In [6]:
#model_id = f"LBK95/llama-7b-qlora-ultrachat"
model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "left"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)
model.config.use_cache = False

print_trainable_parameters(model)


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


test our base model vs fine-tuned model

In [ ]:
do_test = False

if do_test:
    model.eval()
    model.set_adapter("default")
    # test our base model vs fine-tuned model
    text = "### USER: How to plan a picnic?### Assistant:"
    inputs = tokenizer(text, return_tensors="pt").to(0)

    # with adaptor (fine-tuned model)
    model.enable_adapters()
    outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

    print("After attaching Lora adapters:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=False))
    print()

    # without adaptor (base model)
    model.disable_adapters()
    outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

    print("Before attaching Lora adapters:")
    print(tokenizer.decode(outputs[0], skip_special_tokens=False))

    # enable adapters
    model.enable_adapters()

# DPO

Load human prefrence dataset

In [7]:
dataset = load_dataset(
    "lvwerra/stack-exchange-paired",
    split="train[:1%]",
    data_dir="data/rl"
)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def return_prompt_and_responses(samples):
    return {
        "prompt": [
            "Question: " + question + "\n\nAnswer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }

original_columns = dataset.column_names
print(original_columns)

dataset_pref = dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)
print(dataset_pref.column_names)
print(dataset_pref.shape)
print(dataset_pref[1])

['qid', 'question', 'date', 'metadata', 'response_j', 'response_k']


Map:   0%|          | 0/74359 [00:00<?, ? examples/s]

['prompt', 'chosen', 'rejected']
(74359, 3)
{'prompt': 'Question: I\'m writing a code using Java Swing to press the right button when I type a number key.\nBut I can\'t find what I want through search.\nThis is my code and I can\'t understand why this isn\'t working.\nPlease help me..\n\n```\nimport javax.swing.*;\nimport java.awt.Dimension;\nimport java.awt.event.*;\n\nclass class01 {\n\n    public static void main(String[] args) {\n\n        JFrame f = new JFrame("Key event test"); \n        f.setSize(230, 500);\n        f.setLayout(null);\n        f.setVisible(true);\n        f.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);\n\n        JLabel label = new JLabel(); \n\n        JButton button1 = new JButton("Coffe"); \n        button1.setSize(100, 100);\n        button1.setLocation(0, 0);\n\n        JButton button2 = new JButton("Latte");\n        button2.setSize(100, 100);\n        button2.setLocation(0, 100);\n\n        JButton button3 = new JButton("Espresso");\n        button3.set

simple EDA

In [9]:
# very simple eda on the dataset
perform_EDA = False

if perform_EDA:
    df = pd.DataFrame(dataset_pref[:1000])
    df.head()

    df["prompt_length"] = df["prompt"].apply(lambda x: len(x.split()))
    df["chosen_length"] = df["chosen"].apply(lambda x: len(x.split()))
    df["rejected_length"] = df["rejected"].apply(lambda x: len(x.split()))

    df.describe()

    sns.histplot(df["prompt_length"], bins=20)
    plt.show()

    sns.histplot(df["chosen_length"], bins=20)
    plt.show()

    sns.histplot(df["rejected_length"], bins=20)
    plt.show()

training args

In [10]:

# Define training arguments
YOUR_HF_USERNAME = "LBK95" # Set my HF username

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-ultrachat-DPO" # Set output directory for fine-tuned model
per_device_train_batch_size = 4  # Set batch size per device for training
gradient_accumulation_steps = 4  # Set the number of gradient accumulation steps
optim = "paged_adamw_32bit" # Set optimizer
save_steps = 10 # Set the number of steps before saving
logging_steps = 1 # Set the number of steps before logging
learning_rate = 2e-4 # Set learning rate
max_grad_norm = 0.3 # Set max gradient norm
max_steps = 20 # Set max steps
warmup_ratio = 0.03 # Set warmup ratio for learning rate scheduler
lr_scheduler_type = "cosine" # Set learning rate scheduler type


# Create TrainingArguments object with all of our defined arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)

load model ref

In [13]:
model_ref = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
datasef_pref_1000 = dataset_pref.select(range(1000))

create DPO trainer and train

In [20]:
tokenizer.pad_token = tokenizer.eos_token
dpo_trainer = DPOTrainer(
    model,          # base model from SFT pipeline
    model_ref,             # typically a copy of the SFT trained base model
    beta=0.1,              # temperature hyperparameter of DPO
    train_dataset=datasef_pref_1000, # dataset prepared above
    peft_config=lora_config,
    tokenizer=tokenizer,   # tokenizer
    args=training_arguments,    # training arguments e.g. batch size, lr, etc.
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [21]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,29.784900
20,0.000000


TrainOutput(global_step=20, training_loss=14.892425537109375, metrics={'train_runtime': 804.8047, 'train_samples_per_second': 0.398, 'train_steps_per_second': 0.025, 'total_flos': 0.0, 'train_loss': 14.892425537109375, 'epoch': 0.32})

In [22]:
text = "Question: How to plan a picnic?\n\nAnswer: "
inputs = tokenizer(text, return_tensors="pt").to(0)

# with adaptor (fine-tuned model)
#model_loaded.enable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))
print()
# without adaptor (base model)
#model_loaded.disable_adapters()
outputs = model_ref.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


After attaching Lora adapters:
<s> Question: How to plan a picnic?

Answer: <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk